<a href="https://colab.research.google.com/github/cindyhps/analisa_sent_3label/blob/main/analytic_sent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Library

In [ ]:
!pip install gcsfs==2024.9.0

Reason for being yanked: requirements incorrect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2024.10.0
    Uninstalling gcsfs-2024.10.0:
      Successfully uninstalled gcsfs-2024.10.0


In [ ]:
!pip install fsspec==2024.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.9.0 requires fsspec==2024.6.1, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install transformers datasets emoji -q --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00


In [ ]:
!pip install wordcloud matplotlib nltk

In [ ]:
import os
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import emoji
import torch
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
from datasets import Dataset
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score, precision_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


Dataset

In [ ]:
!wget -q https://storage.googleapis.com/babak-dataset/indonlu/smsa_doc-sentiment-prosa/train_preprocess.tsv
!wget -q https://storage.googleapis.com/babak-dataset/indonlu/smsa_doc-sentiment-prosa/valid_preprocess.tsv

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#unzip data
zip_path = '/content/drive/My Drive/indonlu/indonlu-master.zip'
extract_to = '/content/drive/My Drive/indonlu'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

extracted_files = os.listdir(extract_to)
print("Extracted Files:", extracted_files)

Extracted Files: ['indonlu-master.zip', 'indonlu-master', 'trained_model']


In [ ]:
base_path = "/content/drive/My Drive/indonlu/indonlu-master/"
for root, dirs, files in os.walk(base_path):
    print(f"Folder: {root}")
    for file in files:
        print(f"File: {file}")

Folder: /content/drive/My Drive/indonlu/indonlu-master/
File: .gitignore
File: LICENSE
File: README.id.md
File: CONTRIBUTING.md
File: README.md
File: CODE_OF_CONDUCT.md
File: main.py
File: requirements.txt
File: predict.sh
File: predict.py
File: run_all_tasks.sh
File: run_single_task.sh
Folder: /content/drive/My Drive/indonlu/indonlu-master/.github
File: PULL_REQUEST_TEMPLATE.md
File: ISSUE_TEMPLATE.md
Folder: /content/drive/My Drive/indonlu/indonlu-master/data_utils_notebook
File: aspect_based_sentiment_analysis.ipynb
File: ner_data_utils.ipynb
File: aspect_extraction_data_util.ipynb
File: postag_data_utils.ipynb
Folder: /content/drive/My Drive/indonlu/indonlu-master/dataset
Folder: /content/drive/My Drive/indonlu/indonlu-master/dataset/bapos_pos-idn
File: vocab.txt
File: test_preprocess.txt
File: valid_preprocess.txt
File: vocab_uncased.txt
File: test_preprocess_masked_label.txt
File: train_preprocess.txt
Folder: /content/drive/My Drive/indonlu/indonlu-master/dataset/casa_absa-prosa


In [ ]:
from google.colab import files
# Upload file
uploaded = files.upload()

Saving test_preprocess.tsv to test_preprocess.tsv
Saving test_preprocess_masked_label.tsv to test_preprocess_masked_label.tsv
Saving train_preprocess.tsv to train_preprocess.tsv
Saving valid_preprocess.tsv to valid_preprocess.tsv
Saving vocab.txt to vocab.txt
Saving vocab_uncased.txt to vocab_uncased.txt


In [ ]:
print("Current working directory:", os.getcwd())

Current working directory: /content


In [ ]:
print("Files in the current directory:", os.listdir())

Files in the current directory: ['.config', 'valid_preprocess.tsv', 'drive', 'vocab.txt', 'train_preprocess.tsv', 'test_preprocess.tsv', 'vocab_uncased.txt', 'test_preprocess_masked_label.tsv', 'sample_data']


In [ ]:
train_df = pd.read_csv("/content/train_preprocess.tsv", sep="\t", names=["text", "label"])
valid_df = pd.read_csv("/content/valid_preprocess.tsv", sep="\t", names=["text", "label"])

In [ ]:
label_map = {"positive": 0, "neutral": 1, "negative": 2}
train_df["label"] = train_df["label"].map(label_map)
valid_df["label"] = valid_df["label"].map(label_map)

In [ ]:
test_df = pd.read_csv("test_preprocess.tsv", sep="\t", names=["text", "label"])
test_df["label"] = test_df["label"].map(label_map)

Preprocessing Data

In [ ]:
def preprocess_text(text):
    # Convert emoticons to text (emoji description)
    text = emoji.demojize(text, delimiters=(" ", " "))
    # Lowercasing
    text = text.lower()
    return text

In [ ]:
train_df["text"] = train_df["text"].apply(preprocess_text)
valid_df["text"] = valid_df["text"].apply(preprocess_text)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

In [ ]:
test_df["text"] = test_df["text"].apply(preprocess_text)
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.remove_columns(["text"])

NameError: name 'tokenize_data' is not defined

Load Pretrained Model (IndoBERT)

In [ ]:
model_name = "indobenchmark/indobert-base-p1"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

Tokenisasi

In [ ]:
# Tokenisasi dataset
def tokenize_data(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize_data, batched=True)
valid_dataset = valid_dataset.map(tokenize_data, batched=True)

In [ ]:
# Hapus kolom teks yang tidak diperlukan
train_dataset = train_dataset.remove_columns(["text"])
valid_dataset = valid_dataset.remove_columns(["text"])

In [ ]:
# Tentukan data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Training

In [ ]:
training_args = TrainingArguments(
    fp16=True,
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1
)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Saved Model

In [ ]:
trainer.save_model("/content/drive/My Drive/indonlu/trained_model")

Prediksi

In [ ]:
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

Evaluasi

In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
# Evaluasi
results = trainer.evaluate()
print(results)

# Prediksi pada data validasi
predictions = trainer.predict(valid_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Classification report
print(classification_report(valid_dataset["label"], predicted_labels))

# Confusion matrix
cm = confusion_matrix(valid_dataset["label"], predicted_labels)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

Visualisasi

In [ ]:
all_text = " ".join(train_df["text"].tolist())

In [ ]:
kata_hubung = [
    "dan", "atau", "tetapi", "karena", "sehingga", "maka", "jika", "bila", "bahwa",
    "walaupun", "meskipun", "sedangkan", "sambil", "seraya", "lalu", "kemudian",
    "setelah", "sebelum", "sejak", "hingga", "sampai", "supaya", "untuk", "demi",
    "sebab", "oleh karena itu", "padahal", "walau", "apalagi", "bahkan",
    "baik ... maupun ...", "tidak hanya ... tetapi juga ...",
    "bukan hanya ... melainkan juga ...", "agar", "daripada", "tatkala", "ketika",
    "selagi", "asalkan", "seperti", "misalnya", "seandainya", "kendati", "tanpa"
]

In [ ]:
kata_subjek = [
    "saya", "aku", "kami", "kita", "anda", "engkau", "kamu", "kalian",
    "dia", "ia", "mereka", "-nya", "ini", "itu", "tersebut", "sini", "situ",
    "sana", "siapa", "apa", "yang"
]

In [ ]:
def create_wordcloud(text, title):
  stopwords = set(STOPWORDS)
  stopwords.update([kata_subjek, kata_hubung])

  wordcloud = WordCloud(
      width=800,
      height=400,
      background_color="white",
      colormap="viridis",
      max_words=200,
      contour_color="steelblue",
      contour_width=2,
      stopwords=stopwords  # Tambahkan stopwords
  ).generate(all_text)

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Sentiment Data", fontsize=16)
plt.show()

In [ ]:
# Pisahkan teks berdasarkan label
positive_text = " ".join(train_df[train_df["label"] == 0]["text"].tolist())
neutral_text = " ".join(train_df[train_df["label"] == 1]["text"].tolist())
negative_text = " ".join(train_df[train_df["label"] == 2]["text"].tolist())

In [ ]:
kata_hubung = [
    "dan", "atau", "tetapi", "karena", "sehingga", "maka", "jika", "bila", "bahwa",
    "walaupun", "meskipun", "sedangkan", "sambil", "seraya", "lalu", "kemudian",
    "setelah", "sebelum", "sejak", "hingga", "sampai", "supaya", "untuk", "demi",
    "sebab", "oleh karena itu", "padahal", "walau", "apalagi", "bahkan",
    "baik ... maupun ...", "tidak hanya ... tetapi juga ...",
    "bukan hanya ... melainkan juga ...", "agar", "daripada", "tatkala", "ketika",
    "selagi", "asalkan", "seperti", "misalnya", "seandainya", "kendati", "tanpa"
]

In [ ]:
kata_subjek = [
    "saya", "aku", "kami", "kita", "anda", "engkau", "kamu", "kalian",
    "dia", "ia", "mereka", "-nya", "ini", "itu", "tersebut", "sini", "situ",
    "sana", "siapa", "apa", "yang"
]

In [ ]:
def create_wordcloud(text, title):
    # Gabungkan stopwords default dan tambahan
    stopwords_set = set(stopwords.words('indonesian'))
    stopwords_set.update(kata_subjek)
    stopwords_set.update(kata_hubung)

    # Tentukan colormap berdasarkan judul
    if "Positive" in title:
        colormap = "Greens"  # Dominan hijau untuk positif
    elif "Negative" in title:
        colormap = "Reds"    # Dominan merah untuk negatif
    else:
        colormap = "Greys"   # Dominan abu-abu untuk netral

    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color="white",
        colormap=colormap,
        max_words=200,
        contour_color="steelblue",
        contour_width=2,
        stopwords=stopwords_set
    ).generate(text)

    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(title, fontsize=16)
    plt.show()


In [ ]:
create_wordcloud(positive_text, "Word Cloud - Positive Sentiment")
create_wordcloud(neutral_text, "Word Cloud - Neutral Sentiment")
create_wordcloud(negative_text, "Word Cloud - Negative Sentiment")

Test Model

In [ ]:
# Prediksi pada data test
predictions = trainer.predict(test_dataset) # Changed to use test_dataset
predicted_labels = np.argmax(predictions.predictions, axis=1)

In [ ]:
# Evaluasi
accuracy = accuracy_score(test_dataset["label"], predicted_labels)
print(f"Accuracy on test data: {accuracy * 100:.2f}%")